In [2]:
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
file_path=r'D:\MLProjects\DiamondPricePrediction\notebooks\data\gemstone.csv'
df=pd.read_csv(file_path)

In [4]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
df.drop('id',axis=1, inplace=True)

In [6]:
## Segregate Dependent adn Independent features
X=df.drop('price',axis=1)
y=df['price']

In [7]:
## Define which columns should be ordinal coded and which should be scaled
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [8]:
## Define custom ranking for each ordinal variable
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']
color_categories=['D','E','F','G','H','I','J']

In [9]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
## import necessary libraries
from sklearn.impute import SimpleImputer ## to handle missing values
from sklearn.preprocessing import StandardScaler ## Feature Scaling
from sklearn.preprocessing import OrdinalEncoder ##Ordinal Encoding

## To create pipelines
from sklearn.pipeline import Pipeline ## to create a pipeline
from sklearn.compose import ColumnTransformer ## to combine two pipelines


In [11]:
## Creating a numerical pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
        ]
)

## Creating a categorical pipeline
cat_piepline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)

##combining the pipelines
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_piepline,categorical_cols)
])

In [12]:
## Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.30,random_state=30)

In [13]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test= pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [14]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731


In [15]:
## Model Training
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [16]:
import numpy as np
def models(model, X_train, y_train, X_test, y_test):
        regression = model()
        regression.fit(X_train, y_train) 
        predicted=regression.predict(X_test)

        mae = mean_absolute_error(y_test, predicted)
        mse = mean_squared_error(y_test, predicted)
        rmse = np.sqrt(mean_squared_error(y_test, predicted))
        r2_square = r2_score(y_test, predicted)
        
        print('Model Training Performance')
        print('coeff:', regression.coef_)
        print('regression.intercept_:',regression.intercept_)
        print("RMSE:",rmse)
        print("MSE:",mse)
        print("MAE:",mae)
        print("R2 score",r2_square)

In [17]:
model=LinearRegression
models(model, X_train, y_train, X_test, y_test)

Model Training Performance
coeff: [ 6433.66003594  -132.75843566   -70.42922179 -1720.30971463
  -499.29302619   -63.39317848    72.44537247  -460.41604642
   650.76431652]
regression.intercept_: 3970.7662895476774
RMSE: 1013.9047094344005
MSE: 1028002.7598132562
MAE: 674.025511579683
R2 score 0.9368908248567511


In [18]:
model=Ridge
models(model, X_train, y_train, X_test, y_test)

Model Training Performance
coeff: [ 6432.06754964  -132.65578699   -70.45371374 -1713.99205175
  -503.47310729   -63.95924697    72.4292979   -460.37602495
   650.80282483]
regression.intercept_: 3970.7662895476774
RMSE: 1013.905927277163
MSE: 1028005.2293677637
MAE: 674.0555800798246
R2 score 0.9368906732505939


In [19]:
model=Lasso
models(model, X_train, y_train, X_test, y_test)

Model Training Performance
coeff: [ 6389.31239251  -130.29167037   -66.47301825 -2022.96002556
  -173.14068997   -43.93275069    74.25609635  -458.20284703
   650.24723464]
regression.intercept_: 3970.7662895476788
RMSE: 1013.8784226767013
MSE: 1027949.4559693959
MAE: 675.0716923362157
R2 score 0.9368940971841704


In [20]:
model=ElasticNet
models(model, X_train, y_train, X_test, y_test)

Model Training Performance
coeff: [1143.82205687  -14.73896767   10.3742768   774.73272677  775.30866066
  741.0717964    64.13615832  -84.31139233  342.0986292 ]
regression.intercept_: 3970.7662895476788
RMSE: 1533.4162456064046
MSE: 2351365.3822896415
MAE: 1060.7368759154729
R2 score 0.8556494831165182
